Credit to @wenhuchen -  https://github.com/wenhuchen/Program-of-Thoughts

In [1]:
import json
from time import sleep
from tqdm import tqdm
import os
import openai
from datetime import datetime
from tool import finqa_equal, parse_api_result, safe_execute
from eval_tatqa.tatqa_utils import extract_one_num_from_str
from typing import Dict, Any
import argparse
from collections import Counter

In [2]:
def create_reader_request_processed(example: Dict[str, Any]):
    prompt = 'Read the following text and table, and then answer the last question in a series of questions:\n'
    if example['golden_text']:
        prompt += example['golden_text'].strip() + '\n'
    if example['golden_table']:
        prompt += example['golden_table'].strip() + '\n'
    #prompt += '\n'
    prompt += 'Questions: '
    prompt += " ".join(example['questions'][:-1])
    prompt += '\n'
    prompt += f'Question: {example["questions"][-1]}'
    prompt += '\n'
    prompt += 'Answer:'
    return prompt

In [3]:

prompt_4shot = """Read the following text and table, and then answer the last question in a series of questions:
- | shares available for awards | shares subject to outstanding awards
2009 global incentive plan | 2322450 | 2530454
2004 stock incentive plan | - | 5923147
Questions: how many shares are subject to outstanding awards is under the 2009 global incentive plan? what about under the 2004 stock incentive plan? how many total shares are subject to outstanding awards? what about under the 2004 stock incentive plan?
Question: what proportion does this represent?
#Python
shares_subject_to_outstanding_awards_2009_global_incentive_plan = 2530454
shares_subject_to_outstanding_awards_2004_stock_incentive_plan = 5923147
total_shares_subject_to_outstanding_awards = shares_subject_to_outstanding_awards_2009_global_incentive_plan + shares_subject_to_outstanding_awards_2004_stock_incentive_plan
proportion = shares_subject_to_outstanding_awards_2009_global_incentive_plan / total_shares_subject_to_outstanding_awards
ans = proportion


Read the following text and table, and then answer the last question in a series of questions:
compensation expense the company recorded $ 43 million , $ 34 million , and $ 44 million of expense related to stock awards for the years ended december 31 , 2015 , 2014 , and 2013 , respectively . 
Questions: what is the compensation expense the company recorded in 2015? what about in 2014? what is the total compensation expense the company recorded in 2015 and 2014? what is the total expenses including 2013?
Question: what is the average for three years?
#Python
compensation_expense_2015 = 43
compensation_expense_2014 = 34
compensation_expense_2013 = 44
total_compensation_expense = compensation_expense_2015 + compensation_expense_2014 + compensation_expense_2013
average_for_three_years = total_compensation_expense / 3
ans = average_for_three_years


Read the following text and table, and then answer the last question in a series of questions:
the net loss on disposal of those assets was $ 344000 for 2005 and $ 43000 for 2004 . 
Questions: what was the net loss on disposal of assets in 2005? what was the value in 2004? what was the change in value?
Question: what was the percent change?
#Python
net_loss_on_disposal_of_assets_2005 = 344000
net_loss_on_disposal_of_assets_2004 = 43000
net_change_in_value = net_loss_on_disposal_of_assets_2005 - net_loss_on_disposal_of_assets_2004
percent_change = net_change_in_value / net_loss_on_disposal_of_assets_2004
ans = percent_change


Read the following text and table, and then answer the last question in a series of questions:
location | operations conducted | approximatesquare feet | leaseexpirationdates
dublin ireland | global supply chain distribution and administration offices | 160000 | owned
athlone ireland | commercial research and development manufacturing | 80000 | owned
bogart georgia | commercial research and development manufacturing | 70000 | owned
smithfield rhode island | commercial research and development manufacturing | 67000 | owned
Questions: what is the square feet of the owned global supply chain distribution and administration offices? what is the square feet of the owned commercial research and development manufacturing? what is the sum of those values? what is the total sum including square feet of commercial research and development manufacturing in bogart, georgia? what is the total sum including square feet of commercial research and development manufacturing in smithfield, rhode island?
Question: what is the total sum of square feet owned?
#Python
square_feet_owned = [160000, 80000, 70000, 67000]
total_square_feet_owned = sum(square_feet_owned)
ans = total_square_feet_owned
"""

In [11]:
import os
import openai
from configparser import ConfigParser

parser=ConfigParser()
_=parser.read('../../config.cfg')
openai.api_type = "azure"
openai.api_base = parser.get('openai_api','api_ep')
openai.api_version = "2023-03-15-preview"
openai.api_key = parser.get('openai_api','api_key')
model =  parser.get('openai_api','api_model')

os.environ["OPENAI_API_KEY"]  = openai.api_key
os.environ["OPENAI_API_TYPE"] = openai.api_type
os.environ["OPENAI_API_VERSION"] = openai.api_version
os.environ["OPENAI_API_BASE"] = openai.api_base
model_name = "davinci3"

In [7]:
with open('data/convfinqa_dev.json') as f:
        convfinqa_dev = json.load(f)
    
now = datetime.now()
dt_string = now.strftime("%m_%d_%H_%M")

correct, wrong = 0, 0

In [ ]:
filename = f'outputs/convfinqa_direct_{model}_{dt_string}.jsonl'
writer = open(filename, 'w')

for example in tqdm(convfinqa_dev):
    full_prompt = prompt_4shot + "\n\n"
    full_prompt += create_reader_request_processed(example)
    # greedy decoding
    got_result = False
    while not got_result:
        try:
            result = openai.Completion.create(
                engine=model,
                prompt=full_prompt,
                max_tokens=256,
                temperature=0.0,
                top_p=1,
                n=1,
                stop=['\n\n'],
                logprobs=1
            )
            got_result = True
        except Exception:
            sleep(3)
    result_counter = Counter()
    codes = parse_api_result(result)

    for r in codes:
        try :
            exec(r)#extract_one_num_from_str(r)
        except:
            ans=r
        if not ans:
            if 'yes' in r.lower() or 'true' in r.lower():
                ans = 'yes'
            elif 'no' in r.lower() or 'false' in r.lower():
                ans = 'no'
        if ans is not None:
            if type(ans) in [dict]:
                result_counter.update(list(ans.values()))
            elif type(ans) in [list, tuple]:
                result_counter.update([float(ans[0])])
            elif type(ans) in [str]:
                result_counter.update([ans])
            else:
                try:
                    result_counter.update([float(ans)])
                except Exception:
                    continue

    if len(result_counter) > 0:
        prediction = result_counter.most_common(1)[0][0]        
    else:
        prediction = None

    if prediction is None:
        wrong += 1
    elif finqa_equal(prediction, example['answer'], True, True):
        correct += 1
    else:
        wrong += 1

    example.update({'generated': codes, 'executed': prediction})
    writer.write(json.dumps(example) + '\n')
    print('accuracy: ', correct / (correct + wrong))
    
writer.close()


In [10]:
print('accuracy: ', correct / (correct + wrong))


accuracy:  0.6247030878859857
